## 1. Import Libraries và Khởi tạo Spark Session

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import (
    col, explode, split, trim, current_timestamp, row_number, 
    when, regexp_replace, monotonically_increasing_id, dense_rank, udf, collect_list, broadcast
)
from pyspark.sql.window import Window
import os

# Set AWS environment variables for MinIO
os.environ['AWS_REGION'] = 'us-east-1'
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'admin123'

# Khởi tạo Spark Session - SIMPLE CONFIG cho 1 worker x 1.5GB x 2 cores
spark = (
    SparkSession.builder.appName("Load_Data_To_Gold_Tables")
    .master("spark://spark-master:7077")
    
    # === MEMORY CONFIG - Đơn giản cho 1 worker ===
    .config("spark.executor.memory", "1536m")     # 1.5GB
    .config("spark.executor.cores", "2")          # 2 cores
    .config("spark.driver.memory", "512m")
    .config("spark.memory.fraction", "0.6")
    .config("spark.memory.storageFraction", "0.3")
    
    # === SHUFFLE CONFIG ===
    .config("spark.sql.shuffle.partitions", "50")
    .config("spark.default.parallelism", "50")
    
    # === ADAPTIVE QUERY EXECUTION ===
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
    
    # === BROADCAST JOIN ===
    .config("spark.sql.autoBroadcastJoinThreshold", "10MB")
    
    # === COMPRESSION ===
    .config("spark.shuffle.spill.compress", "true")
    .config("spark.shuffle.compress", "true")
    .config("spark.rdd.compress", "true")
    
    # === SERIALIZATION ===
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    
    # ===== Iceberg Catalog qua Nessie =====
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v2")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://gold/")
    .config("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    
    # ===== Cấu hình MinIO =====
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key-id", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .config("spark.sql.catalog.nessie.s3.region", "us-east-1")
    
    # ===== Spark + Hadoop S3 connector =====
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.region", "us-east-1")
    .config("spark.executorEnv.AWS_REGION", "us-east-1")
    .config("spark.executorEnv.AWS_ACCESS_KEY_ID", "admin")
    .config("spark.executorEnv.AWS_SECRET_ACCESS_KEY", "admin123")
    .config("spark.jars", "/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
print("✓ Spark Session - Simple config cho 1 worker!")
print(f"Spark Master: {spark.sparkContext.master}")
print(f"Application ID: {spark.sparkContext.applicationId}")
print(f"Memory: 1.5GB executor + 512MB driver")
print(f"Cores: 2")
print(f"Shuffle Partitions: 50")

25/12/08 07:56:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


✓ Spark Session - Simple config cho 1 worker!
Spark Master: spark://spark-master:7077
Application ID: app-20251208075617-0004
Memory: 1.5GB executor + 512MB driver
Cores: 2
Shuffle Partitions: 50


## 2. Đọc Dữ Liệu từ Result Model và Lọc Records Chưa Xử Lý

In [2]:
# Đọc dữ liệu từ bảng result_multi_model
import gc

df_result = spark.table("nessie.silver_tables.result_multi_model")

print(f"Tổng số records trong result_multi_model: {df_result.count()}")

# Kiểm tra xem Post table đã có dữ liệu chưa
try:
    df_existing_posts = spark.table("nessie.gold_result_model_multi_task.Post")
    existing_count = df_existing_posts.count()
    print(f"Tìm thấy {existing_count} posts đã được xử lý trước đó")
    
    # OPTIMIZE: Lọc bằng join anti thay vì collect() + isin()
    if existing_count > 0:
        df_result_new = df_result.join(
            df_existing_posts.select("postID"),
            on="postID",
            how="left_anti"  # LEFT ANTI JOIN = NOT IN
        )
        new_count = df_result_new.count()
        print(f"Số lượng posts mới cần xử lý: {new_count}")
        
        if new_count == 0:
            print("\n⚠ Không có posts mới để xử lý. Tất cả dữ liệu đã được load vào Gold tables.")
            print("Nếu muốn load lại, hãy xóa dữ liệu trong Gold tables trước.")
        else:
            df_result = df_result_new
    else:
        print("Không có posts nào trong Gold tables, sẽ xử lý toàn bộ dữ liệu")
    
    # Giải phóng df_existing_posts
    df_existing_posts.unpersist()
    del df_existing_posts
    gc.collect()
    
except Exception as e:
    print(f"Gold tables chưa có dữ liệu hoặc chưa tồn tại: {e}")
    print("Sẽ xử lý toàn bộ dữ liệu từ result_multi_model")

# OPTIMIZE: Cache df_result vì sẽ dùng nhiều lần
df_result = df_result.repartition(100).cache()

print("\nSchema:")
df_result.printSchema()
print("\nSample data:")
df_result.show(5, truncate=False)

Tổng số records trong result_multi_model: 3295
Tìm thấy 0 posts đã được xử lý trước đó
Không có posts nào trong Gold tables, sẽ xử lý toàn bộ dữ liệu

Schema:
root
 |-- postID: string (nullable = true)
 |-- timePublish: timestamp (nullable = true)
 |-- description_Normalized: string (nullable = true)
 |-- Label_NER: string (nullable = true)
 |-- Label_Topic: string (nullable = true)
 |-- Label_Intent: string (nullable = true)
 |-- likeCount: integer (nullable = true)
 |-- commentCount: integer (nullable = true)
 |-- shareCount: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- updated_at: timestamp (nullable = true)


Sample data:


+-----------------------------------------------+--------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## 3. Transform và Load vào Table Entity

Entity table chỉ lưu các loại entity types và mô tả (VD: MAJOR - "Chuyên ngành học")

In [3]:
# Định nghĩa mô tả cho các entity types
import gc

entity_type_descriptions = {
    "ORG": "ORGANIZATION : Tổ chức/Trường (ví dụ: Trường Đại học Bách khoa Hà Nội, Viện Công nghệ, các viện/khoa/bộ môn)",
    "PRO": "PROGRAM : Chương trình đào tạo (ví dụ: Chương trình Tiên tiến, Chương trình KSCLC, ELITECH)",
    "MAJ": "MAJOR : Ngành học, chuyên ngành, nghề nghiệp (ví dụ: Khoa học máy tính, Công nghệ thông tin, Kỹ thuật Điện tử - Viễn thông, Bác sĩ)",
    "SCO": "SCORE : Điểm số (ví dụ: 27.5 điểm, Điểm chuẩn 28)",
    "DATE": "DATE : Thời gian (ví dụ: Năm 2024, Năm 2025, 15/8/2024)",
    "EX": "EXAM : Kỳ thi (ví dụ: Kỳ thi đánh giá năng lực, TOEIC, IELTS, TOPIK)",
    "LOC": "LOCATION : Địa điểm (ví dụ: Hà Nội, Thành phố Hồ Chí Minh)",
    "FEE": "FEE : Học phí/Chi phí (ví dụ: Học phí, Mức đóng, Học bổng)",
    "SUBJ": "SUBJECT : Môn học (ví dụ: Toán, Lý, Hóa)",
    "TERM": "TERM : Tổ hợp môn (ví dụ: A1, B1)",
    "SAL": "SALARY : Lương (ví dụ: Mức lương, Thu nhập)"
}

# Parse NER labels để extract entity types
@udf(returnType=ArrayType(StringType()))
def extract_entity_types(ner_labels):
    """Extract unique entity types from NER labels"""
    if not ner_labels:
        return []
    
    labels = ner_labels.split()
    entity_types = set()
    
    for label in labels:
        if label.startswith('B-') or label.startswith('I-'):
            entity_type = label[2:]  # Lấy phần sau "B-" hoặc "I-"
            entity_types.add(entity_type)
    
    return list(entity_types)

# Extract unique entity types từ toàn bộ dữ liệu
df_entity_types_raw = df_result.select(
    explode(extract_entity_types(col("Label_NER"))).alias("entityType")
).filter(
    col("entityType").isNotNull()
).distinct()

# Tạo DataFrame với entityDetail từ mapping
from pyspark.sql.functions import lit

# Tạo list các entity types với descriptions
entity_list = []
for entity_type in df_entity_types_raw.collect():
    et = entity_type.entityType
    entity_list.append({
        "entityType": et,
        "entityDetail": entity_type_descriptions.get(et, f"Entity type: {et}")
    })

if entity_list:
    df_unique_entities = spark.createDataFrame(entity_list)
    
    # Thêm entityID
    window_spec = Window.orderBy("entityType")
    df_entities = df_unique_entities.withColumn(
        "entityID",
        row_number().over(window_spec)
    ).withColumn(
        "created_at", current_timestamp()
    ).withColumn(
        "updated_at", current_timestamp()
    ).select(
        col("entityID"),
        col("entityType"),
        col("entityDetail"),
        col("created_at"),
        col("updated_at")
    )
    
    print(f"\nSố lượng unique entity types từ dữ liệu mới: {df_entities.count()}")
    df_entities.show(20, truncate=False)
    
    # Kiểm tra và merge với entities đã có
    print("\n=== Loading data into Entity table ===")
    try:
        df_existing_entities = spark.table("nessie.gold_result_model_multi_task.Entity")
        existing_count = df_existing_entities.count()
        print(f"Tìm thấy {existing_count} entity types đã tồn tại")
        
        max_entity_id = df_existing_entities.agg({"entityID": "max"}).collect()[0][0]
        if max_entity_id is None:
            max_entity_id = 0
        print(f"Max entityID hiện tại: {max_entity_id}")
        
        # Chỉ thêm các entity types mới
        df_new_entities = df_entities.join(
            df_existing_entities.select("entityType"),
            on="entityType",
            how="left_anti"
        )
        
        new_count = df_new_entities.count()
        print(f"Số lượng entity types mới cần thêm: {new_count}")
        
        if new_count > 0:
            window_spec = Window.orderBy("entityType")
            df_new_entities = df_new_entities.withColumn(
                "entityID",
                row_number().over(window_spec) + max_entity_id
            ).select(
                col("entityID"),
                col("entityType"),
                col("entityDetail"),
                col("created_at"),
                col("updated_at")
            )
            
            df_new_entities.writeTo("nessie.gold_result_model_multi_task.Entity") \
                .using("iceberg") \
                .append()
            print(f"Đã thêm {new_count} entity types mới!")
            
            df_entities = df_existing_entities.union(df_new_entities)
            
            # Giải phóng df_new_entities
            df_new_entities.unpersist()
            del df_new_entities
        else:
            print("Không có entity types mới cần thêm")
            df_entities = df_existing_entities
        
        # Giải phóng df_existing_entities
        df_existing_entities.unpersist()
        del df_existing_entities
            
    except Exception as e:
        print(f"Table Entity chưa có dữ liệu hoặc chưa tồn tại: {e}")
        print("Tạo mới table với dữ liệu hiện tại")
        df_entities.writeTo("nessie.gold_result_model_multi_task.Entity") \
            .using("iceberg") \
            .create()
        print(f"Đã tạo table Entity với {df_entities.count()} records!")
    
    # Giải phóng các DataFrame tạm
    df_entity_types_raw.unpersist()
    df_unique_entities.unpersist()
    del df_entity_types_raw, df_unique_entities
    gc.collect()
    print("✓ Đã giải phóng bộ nhớ")
else:
    print("Không tìm thấy entity types trong dữ liệu")


Số lượng unique entity types từ dữ liệu mới: 12


+--------+----------+-----------------------------------------------------------------------------------------------------------------------------------+--------------------------+--------------------------+
|entityID|entityType|entityDetail                                                                                                                       |created_at                |updated_at                |
+--------+----------+-----------------------------------------------------------------------------------------------------------------------------------+--------------------------+--------------------------+
|1       |DATE      |DATE : Thời gian (ví dụ: Năm 2024, Năm 2025, 15/8/2024)                                                                            |2025-12-08 07:57:13.388097|2025-12-08 07:57:13.388097|
|2       |EX        |EXAM : Kỳ thi (ví dụ: Kỳ thi đánh giá năng lực, TOEIC, IELTS, TOPIK)                                                               |2025-12-08 07:5

Số lượng entity types mới cần thêm: 12


Đã thêm 12 entity types mới!
✓ Đã giải phóng bộ nhớ


## 4. Transform và Load vào Table Topic

Topic table lưu các chủ đề và mô tả (VD: MAJOR - "Ngành học, chuyên ngành")

In [4]:
# Định nghĩa mô tả cho các topic types
import gc

topic_descriptions = {
    "MAJOR": "Ngành học, chuyên ngành",
    "SUBJECT_COMBINATION": "Tổ hợp môn thi (A, B, C, D)",
    "TUITION": "Học phí, học bổng",
    "CERTIFICATE": "IELTS, TOEFL, SAT, kỳ thi năng lực",
    "UNIVERSITY": "Thông tin trường",
    "STUDENT_LIFE": "Ký túc xá, CLB, làm thêm",
    "CAREER": "Nghề nghiệp, lương",
    "STUDY": "Phương pháp học, môn học ĐH",
    "LANGUAGE": "Tiếng Anh, ngoại ngữ",
    "OTHER": "Khác"
}

# Parse Label_Topic (multi-label, phân cách bằng '|')
df_topics_raw = df_result.select(
    explode(split(col("Label_Topic"), "\\|")).alias("topicName")
).filter(
    (col("topicName").isNotNull()) & 
    (col("topicName") != "None") &
    (trim(col("topicName")) != "")
).select(
    trim(col("topicName")).alias("topicName")
).distinct()

# Tạo list các topics với descriptions
topic_list = []
for topic in df_topics_raw.collect():
    topic_name = topic.topicName
    topic_list.append({
        "topicName": topic_name,
        "topicDetail": topic_descriptions.get(topic_name, "Chủ đề khác")
    })

if topic_list:
    df_unique_topics = spark.createDataFrame(topic_list)
    
    # Thêm topicID
    window_spec = Window.orderBy("topicName")
    df_topics = df_unique_topics.withColumn(
        "topicID",
        row_number().over(window_spec)
    ).withColumn(
        "created_at", current_timestamp()
    ).withColumn(
        "updated_at", current_timestamp()
    ).select(
        col("topicID"),
        col("topicName"),
        col("topicDetail"),
        col("created_at"),
        col("updated_at")
    )
    
    print(f"Số lượng unique topics từ dữ liệu mới: {df_topics.count()}")
    df_topics.show(20, truncate=False)
    
    print("\n=== Loading data into Topic table ===")
    try:
        df_existing_topics = spark.table("nessie.gold_result_model_multi_task.Topic")
        existing_count = df_existing_topics.count()
        print(f"Tìm thấy {existing_count} topics đã tồn tại")
        
        max_topic_id = df_existing_topics.agg({"topicID": "max"}).collect()[0][0]
        if max_topic_id is None:
            max_topic_id = 0
        print(f"Max topicID hiện tại: {max_topic_id}")
        
        # Chỉ thêm các topics mới
        df_new_topics = df_topics.join(
            df_existing_topics.select("topicName"),
            on="topicName",
            how="left_anti"
        )
        
        new_count = df_new_topics.count()
        print(f"Số lượng topics mới cần thêm: {new_count}")
        
        if new_count > 0:
            window_spec = Window.orderBy("topicName")
            df_new_topics = df_new_topics.withColumn(
                "topicID",
                row_number().over(window_spec) + max_topic_id
            ).select(
                col("topicID"),
                col("topicName"),
                col("topicDetail"),
                col("created_at"),
                col("updated_at")
            )
            
            df_new_topics.writeTo("nessie.gold_result_model_multi_task.Topic") \
                .using("iceberg") \
                .append()
            print(f"Đã thêm {new_count} topics mới!")
            
            df_topics = df_existing_topics.union(df_new_topics)
            
            # Giải phóng df_new_topics
            df_new_topics.unpersist()
            del df_new_topics
        else:
            print("Không có topics mới cần thêm")
            df_topics = df_existing_topics
        
        # Giải phóng df_existing_topics
        df_existing_topics.unpersist()
        del df_existing_topics
            
    except Exception as e:
        print(f"Table Topic chưa có dữ liệu hoặc chưa tồn tại: {e}")
        print("Tạo mới table với dữ liệu hiện tại")
        df_topics.writeTo("nessie.gold_result_model_multi_task.Topic") \
            .using("iceberg") \
            .create()
        print(f"Đã tạo table Topic với {df_topics.count()} records!")
    
    # Giải phóng các DataFrame tạm
    df_topics_raw.unpersist()
    df_unique_topics.unpersist()
    del df_topics_raw, df_unique_topics
    gc.collect()
    print("✓ Đã giải phóng bộ nhớ")
else:
    print("Không tìm thấy topics trong dữ liệu")

Số lượng unique topics từ dữ liệu mới: 10


+-------+-------------------+----------------------------------+--------------------------+--------------------------+
|topicID|topicName          |topicDetail                       |created_at                |updated_at                |
+-------+-------------------+----------------------------------+--------------------------+--------------------------+
|1      |CAREER             |Nghề nghiệp, lương                |2025-12-08 07:57:22.208409|2025-12-08 07:57:22.208409|
|2      |CERTIFICATE        |IELTS, TOEFL, SAT, kỳ thi năng lực|2025-12-08 07:57:22.208409|2025-12-08 07:57:22.208409|
|3      |LANGUAGE           |Tiếng Anh, ngoại ngữ              |2025-12-08 07:57:22.208409|2025-12-08 07:57:22.208409|
|4      |MAJOR              |Ngành học, chuyên ngành           |2025-12-08 07:57:22.208409|2025-12-08 07:57:22.208409|
|5      |OTHER              |Khác                              |2025-12-08 07:57:22.208409|2025-12-08 07:57:22.208409|
|6      |STUDENT_LIFE       |Ký túc xá, CLB, làm

Số lượng topics mới cần thêm: 10


Đã thêm 10 topics mới!
✓ Đã giải phóng bộ nhớ


## 5. Transform và Load vào Table Post

In [5]:
from pyspark.sql.functions import col, current_timestamp
import math
import time
import gc

# --- CAU HINH ---
BATCH_SIZE = 1000  # Xu ly 1000 dong moi lan

print(f"=== CHAY VOI CAU HINH: 3 WORKERS | BATCH: {BATCH_SIZE} ===")

# 1. CHUAN BI DATA
# Lay nguyen goc du lieu (khong cat chuoi vi du lieu < 256 ky tu)
df_posts_final = df_result.select(
    col("postID"),
    col("description_Normalized").alias("description"),
    col("timePublish"),
    col("likeCount"),
    col("commentCount"),
    col("shareCount"),
    col("Label_Intent").alias("intent"),
    col("type")
).withColumn("created_at", current_timestamp()) \
 .withColumn("updated_at", current_timestamp())

# 2. LAY LIST ID DE CHIA VIEC
all_ids = [r.postID for r in df_posts_final.select("postID").collect()]
total_records = len(all_ids)
print(f"Tong so bai can xu ly: {total_records}")

if total_records > 0:
    # 3. KHOI TAO BANG (neu chua co)
    try:
        # Lay 1 dong mau de tao bang
        df_posts_final.limit(1).writeTo("nessie.gold_result_model_multi_task.Post") \
            .using("iceberg").createOrReplace()
        print("Da khoi tao bang dich.")
    except Exception as e:
        print("Bang da ton tai, chuyen sang che do Append.")

    # 4. CHAY VONG LAP (BATCH PROCESSING)
    num_batches = math.ceil(total_records / BATCH_SIZE)
    
    for i in range(num_batches):
        start = i * BATCH_SIZE
        end = start + BATCH_SIZE
        batch_ids = all_ids[start:end]
        
        print(f"Batch {i+1}/{num_batches}: Dang xu ly {len(batch_ids)} dong...")
        
        # Loc du lieu cho batch nay
        df_batch = df_posts_final.filter(col("postID").isin(batch_ids))
        
        # Repartition = 6 (toi uu cho 3 workers x 2 cores)
        try:
            df_batch.repartition(6).writeTo("nessie.gold_result_model_multi_task.Post") \
                .using("iceberg") \
                .append()
            print(f"   Batch {i+1} OK.")
        except Exception as e:
            print(f"   Batch {i+1} LOI: {e}")
            
        # Don dep RAM
        df_batch.unpersist()
        del df_batch
        gc.collect()

    print("\nHOAN TAT! Du lieu da duoc ghi thanh cong.")
    
    # Giải phóng df_posts_final
    df_posts_final.unpersist()
    del df_posts_final, all_ids
    gc.collect()
    print("✓ Đã giải phóng bộ nhớ")
else:
    print("Khong co du lieu moi.")

=== CHAY VOI CAU HINH: 3 WORKERS | BATCH: 1000 ===
Tong so bai can xu ly: 3295
Da khoi tao bang dich.
Batch 1/4: Dang xu ly 1000 dong...
   Batch 1 OK.
Batch 2/4: Dang xu ly 1000 dong...
   Batch 2 OK.
Batch 3/4: Dang xu ly 1000 dong...
   Batch 3 OK.
Batch 4/4: Dang xu ly 295 dong...
   Batch 4 OK.

HOAN TAT! Du lieu da duoc ghi thanh cong.
✓ Đã giải phóng bộ nhớ


## 6. Transform và Load vào Table Post_Entity

Bảng quan hệ M:N giữa Post và Entity - Lưu các entity values thực tế được extract từ posts

In [6]:
from pyspark.sql import Window
from pyspark.sql.functions import (
    udf, col, explode, row_number, current_timestamp, 
    trim, lower, regexp_replace, when, count, broadcast, lit
)
from pyspark.sql.types import ArrayType, StructType, StructField, StringType
import gc

# ==============================================================================
# 1. DEFINE UDF (User Defined Function)
# ==============================================================================
@udf(returnType=ArrayType(StructType([
    StructField("entityType", StringType()),
    StructField("entityName", StringType())
])))
def extract_entities_raw(ner_labels, text):
    """
    Extract entities từ BIO labels và text.
    """
    if not ner_labels or not text:
        return []
    
    # Clean nhẹ và split
    labels = ner_labels.strip().split()
    words = text.strip().split()
    
    # Kiểm tra độ dài để tránh lỗi index out of range
    if len(labels) != len(words):
        return []
    
    entities = []
    current_type = None
    current_words = []
    
    for label, word in zip(labels, words):
        label = label.strip()
        
        if label.startswith('B-'):
            if current_type and current_words:
                entities.append({'entityType': current_type, 'entityName': ' '.join(current_words)})
            current_type = label[2:]
            current_words = [word]
            
        elif label.startswith('I-') and current_type:
            # Strict check: I-tag phải khớp với B-tag
            if label[2:] == current_type:
                current_words.append(word)
            else:
                entities.append({'entityType': current_type, 'entityName': ' '.join(current_words)})
                current_type = None
                current_words = []
        else:
            if current_type and current_words:
                entities.append({'entityType': current_type, 'entityName': ' '.join(current_words)})
            current_type = None
            current_words = []
    
    if current_type and current_words:
        entities.append({'entityType': current_type, 'entityName': ' '.join(current_words)})
    
    return entities

# ==============================================================================
# 2. MAIN PROCESSING FLOW
# ==============================================================================

print("=== Extracting entities từ posts ===")

# --- Bước A: Extract & Clean Data ---
# Giả định df_result đã có sẵn từ các bước trước
df_extracted = df_result.select(
    col("postID"),
    col("Label_NER"),
    col("description_Normalized")
).select(
    col("postID"),
    explode(extract_entities_raw(col("Label_NER"), col("description_Normalized"))).alias("entity")
).select(
    col("postID"),
    col("entity.entityType").alias("entityType"),
    col("entity.entityName").alias("raw_name")
).withColumn(
    "entityName",
    regexp_replace(col("raw_name"), "_", " ")
).withColumn(
    "entityName",
    regexp_replace(col("entityName"), r"^\.+|\.+$", "")
).withColumn(
    "entityName",
    trim(col("entityName"))
).filter(
    (col("entityType").isNotNull()) & (col("entityName") != "")
).drop("raw_name")

# Repartition và cache để tối ưu hiệu suất cho các bước sau
df_extracted = df_extracted.repartition(200, "entityType").cache()

extracted_count = df_extracted.count()
print(f"✓ Entities extracted (cleaned): {extracted_count}")

if extracted_count == 0:
    print("⚠ Không có entities để xử lý!")
else:
    # --- Bước B: Add Entity Order ---
    window_spec = Window.partitionBy("postID").orderBy("entityType", "entityName")
    df_ordered = df_extracted.withColumn(
        "entityOrder",
        row_number().over(window_spec)
    )
    
    # --- Bước C: Chuẩn bị dữ liệu để JOIN ---
    # Chuẩn bị bảng Entities (Master Data)
    # Distinct để đảm bảo 1 Type không xuất hiện nhiều lần gây nhân bản dữ liệu
    df_entities_ready = broadcast(
        df_entities.select(
            col("entityID"),
            col("entityType"),
            lower(trim(col("entityType"))).alias("type_norm")
        ).distinct() 
    )
    
    # Chuẩn bị bảng Post (Transaction Data)
    df_post_ready = df_ordered.withColumn(
        "type_norm", lower(trim(col("entityType")))
    )
    
    # --- Bước D: JOIN ---
    # Thêm cột entityNameNormalized với giá trị NULL
    df_final_join = df_post_ready.join(
        df_entities_ready,
        on="type_norm",
        how="inner"
    ).select(
        col("postID"),
        col("entityID"),
        col("entityOrder"),
        col("entityName"),
        lit(None).cast(StringType()).alias("entityNameNormalized")  # Thêm cột mới với giá trị NULL
    ).withColumn(
        "created_at", current_timestamp()
    ).withColumn(
        "updated_at", current_timestamp()
    )
    
    final_count = df_final_join.count()
    print(f"✓ Số lượng quan hệ Post-Entity: {final_count}")
    
    if final_count > 0:
        # Tối ưu kích thước file Iceberg (Tránh small files problem)
        # Giả sử target 100k - 200k dòng mỗi file
        num_partitions = max(1, final_count // 100000) 
        df_final_join = df_final_join.coalesce(num_partitions)
        
        # --- WRITE TO ICEBERG ---
        target_table = "nessie.gold_result_model_multi_task.Post_Entity"
        
        try:
            print(f"\n Đang ghi {final_count} records vào {target_table}...")
            df_final_join.writeTo(target_table) \
                .using("iceberg") \
                .append()
            print(f"✓ SUCCESS: Đã thêm {final_count} dòng vào bảng Post_Entity.")
            
        except Exception as e:
            # Xử lý trường hợp bảng chưa tồn tại
            if "not found" in str(e).lower() or "does not exist" in str(e).lower():
                print(f"Table {target_table} chưa tồn tại. Đang tạo mới...")
                df_final_join.writeTo(target_table) \
                    .using("iceberg") \
                    .create()
                print(f"✓ SUCCESS: Đã tạo bảng và insert {final_count} dòng.")
            else:
                print(" ERROR: Ghi dữ liệu thất bại.")
                # In ra schema để debug nếu vẫn lỗi
                print("Schema DataFrame hiện tại:")
                df_final_join.printSchema()
                raise e
    else:
        print("⚠ WARNING: Không có dữ liệu sau khi Join (Check lại bảng df_entities có map đúng Type không).")
    
    # Cleanup memory
    df_extracted.unpersist()
    df_ordered.unpersist()
    df_final_join.unpersist()
    del df_extracted, df_ordered, df_post_ready, df_final_join
    gc.collect()
    print("\n✓ Đã giải phóng bộ nhớ cache.")

=== Extracting entities từ posts ===


✓ Entities extracted (cleaned): 16744


✓ Số lượng quan hệ Post-Entity: 16744

 Đang ghi 16744 records vào nessie.gold_result_model_multi_task.Post_Entity...


✓ SUCCESS: Đã thêm 16744 dòng vào bảng Post_Entity.

✓ Đã giải phóng bộ nhớ cache.


## 7. Transform và Load vào Table Post_Topic

Bảng quan hệ M:N giữa Post và Topic

In [7]:
# Parse topics cho mỗi post
import gc

df_post_topics_raw = df_result.select(
    col("postID"),
    explode(split(col("Label_Topic"), "\\|")).alias("topicName")
).filter(
    (col("topicName").isNotNull()) & 
    (col("topicName") != "None") &
    (trim(col("topicName")) != "")
).select(
    col("postID"),
    trim(col("topicName")).alias("topicName")
).distinct()

# OPTIMIZE: Broadcast Topic table vì nhỏ
df_post_topic = df_post_topics_raw.join(
    broadcast(df_topics.select("topicID", "topicName")),
    on="topicName",
    how="inner"
).select(
    col("postID"),
    col("topicID")
).withColumn(
    "created_at", current_timestamp()
).withColumn(
    "updated_at", current_timestamp()
)

topic_count = df_post_topic.count()
print(f"Tổng số quan hệ Post-Topic mới: {topic_count}")
df_post_topic.show(10, truncate=False)

print("\n=== Loading data into Post_Topic table ===")
if topic_count > 0:
    # OPTIMIZE: Coalesce để giảm số partitions
    num_partitions = max(1, topic_count // 1000)
    df_post_topic = df_post_topic.coalesce(num_partitions)
    
    try:
        df_post_topic.writeTo("nessie.gold_result_model_multi_task.Post_Topic") \
            .using("iceberg") \
            .append()
        print(f"✓ Đã thêm {topic_count} quan hệ Post-Topic mới!")
    except Exception as e:
        if "table does not exist" in str(e).lower() or "not found" in str(e).lower():
            print("Table Post_Topic chưa tồn tại, tạo mới...")
            df_post_topic.writeTo("nessie.gold_result_model_multi_task.Post_Topic") \
                .using("iceberg") \
                .create()
            print(f"✓ Đã tạo table Post_Topic với {topic_count} records!")
        else:
            raise e
else:
    print("Không có quan hệ Post-Topic mới cần thêm")

# Giải phóng bộ nhớ
df_post_topics_raw.unpersist()
df_post_topic.unpersist()
del df_post_topics_raw, df_post_topic
gc.collect()
print("✓ Đã giải phóng bộ nhớ")

Tổng số quan hệ Post-Topic mới: 5007


+----------------------------------------+-------+--------------------------+--------------------------+
|postID                                  |topicID|created_at                |updated_at                |
+----------------------------------------+-------+--------------------------+--------------------------+
|@marketinghaui/video/7260473091074084097|10     |2025-12-08 07:57:47.113145|2025-12-08 07:57:47.113145|
|2048882745968831                        |3      |2025-12-08 07:57:47.113145|2025-12-08 07:57:47.113145|
|2061475374709568                        |1      |2025-12-08 07:57:47.113145|2025-12-08 07:57:47.113145|
|2081996429324129                        |10     |2025-12-08 07:57:47.113145|2025-12-08 07:57:47.113145|
|1911718409685266                        |8      |2025-12-08 07:57:47.113145|2025-12-08 07:57:47.113145|
|1790515065138935                        |4      |2025-12-08 07:57:47.113145|2025-12-08 07:57:47.113145|
|@bikipduhoc/video/7465322815005199624   |10     |2025-

✓ Đã thêm 5007 quan hệ Post-Topic mới!
✓ Đã giải phóng bộ nhớ


In [8]:
# === MEMORY CLEANUP ===
import gc

print("\n=== Cleaning up memory ===")
df_result.unpersist()
spark.catalog.clearCache()

# Giải phóng các biến còn lại
try:
    del df_entities, df_topics
except:
    pass

gc.collect()
print("✓ Đã giải phóng toàn bộ bộ nhớ cache.")


=== Cleaning up memory ===
✓ Đã giải phóng toàn bộ bộ nhớ cache.


## 8. Verify Dữ Liệu Đã Load

In [9]:
import gc

print("="*80)
print("SUMMARY - Data Loaded to Gold Tables")
print("="*80)

tables = [
    "Entity",
    "Topic",
    "Post",
    "Post_Entity",
    "Post_Topic"
]

for table_name in tables:
    df_verify = spark.table(f"nessie.gold_result_model_multi_task.{table_name}")
    count = df_verify.count()
    print(f"\n{'='*80}")
    print(f"Table: {table_name}")
    print(f"Total records: {count}")
    print(f"{'='*80}")
    df_verify.show(5, truncate=False)
    
    # Giải phóng từng DataFrame sau khi show
    df_verify.unpersist()
    del df_verify
    gc.collect()

print("\n" + "="*80)
print("Hoàn thành load dữ liệu vào tất cả Gold tables!")
print("="*80)
print("✓ Đã giải phóng bộ nhớ")

SUMMARY - Data Loaded to Gold Tables

Table: Entity
Total records: 12
+--------+----------+-----------------------------------------------------------------------------------------------------------------------------------+--------------------------+--------------------------+
|entityID|entityType|entityDetail                                                                                                                       |created_at                |updated_at                |
+--------+----------+-----------------------------------------------------------------------------------------------------------------------------------+--------------------------+--------------------------+
|1       |DATE      |DATE : Thời gian (ví dụ: Năm 2024, Năm 2025, 15/8/2024)                                                                            |2025-12-08 07:57:17.326012|2025-12-08 07:57:17.326012|
|2       |EX        |EXAM : Kỳ thi (ví dụ: Kỳ thi đánh giá năng lực, TOEIC, IELTS, TOPIK)         

## 9. Thống Kê và Phân Tích

In [10]:
# ==============================================================================
# DATA PROFILING & STATISTICS (FIXED COLUMN NAMES)
# ==============================================================================

# 1. Thống kê Entity Types và Descriptions
print("\n=== 1. Entity Types và Descriptions ===")
spark.sql("""
    SELECT entityType, entityDetail
    FROM nessie.gold_result_model_multi_task.Entity
    ORDER BY entityType
""").show(truncate=False)

# 2. Thống kê số lượng entity mentions theo type (Top 20)
print("\n=== 2. Entity Mentions Distribution by Type (Top 20) ===")
spark.sql("""
    SELECT e.entityType, e.entityDetail, COUNT(pe.postID) as mention_count
    FROM nessie.gold_result_model_multi_task.Entity e
    LEFT JOIN nessie.gold_result_model_multi_task.Post_Entity pe ON e.entityID = pe.entityID
    GROUP BY e.entityType, e.entityDetail
    ORDER BY mention_count DESC
    LIMIT 20
""").show(truncate=False)

# 3. Thống kê Topics
print("\n=== 3. Topics và Descriptions ===")
spark.sql("""
    SELECT topicName, topicDetail
    FROM nessie.gold_result_model_multi_task.Topic
    ORDER BY topicName
""").show(truncate=False)

# 4. Thống kê số lượng posts theo topic
print("\n=== 4. Topic Distribution ===")
spark.sql("""
    SELECT t.topicName, t.topicDetail, COUNT(pt.postID) as post_count
    FROM nessie.gold_result_model_multi_task.Topic t
    LEFT JOIN nessie.gold_result_model_multi_task.Post_Topic pt ON t.topicID = pt.topicID
    GROUP BY t.topicName, t.topicDetail
    ORDER BY post_count DESC
""").show(20, truncate=False)

# 5. Thống kê intent distribution
print("\n=== 5. Intent Distribution ===")
spark.sql("""
    SELECT intent, COUNT(*) as count
    FROM nessie.gold_result_model_multi_task.Post
    GROUP BY intent
    ORDER BY count DESC
""").show(truncate=False)

# 6. Top entity values được nhắc đến nhiều nhất
# FIX: Đã đổi entityValue -> entityName
print("\n=== 6. Top 20 Most Mentioned Entity Values ===")
spark.sql("""
    SELECT 
        e.entityType,
        pe.entityName,  -- <--- SỬA LẠI TÊN CỘT Ở ĐÂY
        COUNT(DISTINCT pe.postID) as mention_count
    FROM nessie.gold_result_model_multi_task.Post_Entity pe
    JOIN nessie.gold_result_model_multi_task.Entity e ON pe.entityID = e.entityID
    GROUP BY e.entityType, pe.entityName -- <--- VÀ Ở ĐÂY
    ORDER BY mention_count DESC
    LIMIT 20
""").show(truncate=False)

# 7. Sample Entity Values by Type (5 samples per type)
# FIX: Đã đổi entityValue -> entityName
print("\n=== 7. Sample Entity Values by Type (5 samples per type) ===")
spark.sql("""
    WITH RankedValues AS (
        SELECT 
            e.entityType,
            e.entityDetail,
            pe.entityName, -- <--- SỬA LẠI TÊN CỘT Ở ĐÂY
            ROW_NUMBER() OVER (PARTITION BY e.entityType ORDER BY pe.entityName) as rn
        FROM nessie.gold_result_model_multi_task.Entity e
        LEFT JOIN nessie.gold_result_model_multi_task.Post_Entity pe ON e.entityID = pe.entityID
        WHERE pe.entityName IS NOT NULL
    )
    SELECT 
        entityType,
        entityDetail,
        entityName as sample_value
    FROM RankedValues
    WHERE rn <= 5
    ORDER BY entityType, rn
""").show(50, truncate=False)


=== 1. Entity Types và Descriptions ===
+----------+-----------------------------------------------------------------------------------------------------------------------------------+
|entityType|entityDetail                                                                                                                       |
+----------+-----------------------------------------------------------------------------------------------------------------------------------+
|DATE      |DATE : Thời gian (ví dụ: Năm 2024, Năm 2025, 15/8/2024)                                                                            |
|EX        |EXAM : Kỳ thi (ví dụ: Kỳ thi đánh giá năng lực, TOEIC, IELTS, TOPIK)                                                               |
|FEE       |FEE : Học phí/Chi phí (ví dụ: Học phí, Mức đóng, Học bổng)                                                                         |
|LOC       |LOCATION : Địa điểm (ví dụ: Hà Nội, Thành phố Hồ Chí Minh)                   

## 10. Dừng Spark Session

In [11]:
# Giải phóng bộ nhớ trước khi dừng Spark
import gc

# Xóa tất cả cache
spark.catalog.clearCache()

# Thu gom rác
gc.collect()
print("✓ Đã giải phóng toàn bộ bộ nhớ")

# Dừng Spark Session
spark.stop()
print("✓ Spark Session đã được dừng!")

✓ Đã giải phóng toàn bộ bộ nhớ
✓ Spark Session đã được dừng!
